In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_90.csv', index_col=0)

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=1000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
# sen1Bow = dictionary.doc2bow(df['reduced_tokens'][0])
# sen1lda = senlda[sen1Bow]
# print("The topics of the text: {}".format(df['TI'][0]))
# print("are: {}".format(sen1lda))

In [11]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [12]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [13]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,patient,health,disease,work,risk,physical,child,age,study,health,patient,study,health,study,determinant,health,health,age,study,determinant
1,study,age,study,self,health,activity,behavior,health,child,condom,study,subject,patient,treatment,study,patient,support,year,use,factor
2,factor,factor,health,model,age,health,factor,use,health,use,use,increase,determinant,result,health,use,determinant,determinant,factor,rate
3,disability,risk,patient,influence,study,increase,health,support,factor,care,result,group,self,patient,age,determinant,self,group,age,control
4,handicap,study,high,behavior,high,determinant,study,factor,use,determinant,determinant,variable,study,determinant,factor,child,child,population,result,community
5,use,year,determinant,low,breastfeeding,age,female,family,determinant,gender,factor,factor,care,age,population,disease,care,care,population,birth
6,influence,child,risk,result,time,group,determinant,study,effect,age,research,female,life,year,result,study,result,study,condom,result
7,risk,level,report,information,rate,study,age,cancer,status,sex,change,age,research,smoking,effect,factor,physical,high,determinant,high
8,population,high,result,determinant,year,cancer,influence,model,age,study,analysis,male,factor,rate,measure,status,knowledge,patient,behaviour,study
9,disease,status,care,job,support,rate,analysis,determinant,variable,influence,disease,hiv,practice,measure,level,drug,high,depression,consumption,health


In [14]:
import pyLDAvis.gensim
lda_model = senlda
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.031546  0.016849       1        1  8.686917
8     -0.004238 -0.001143       2        1  8.681153
2     -0.005099  0.001906       3        1  7.500094
9      0.029093  0.028260       4        1  6.964219
1      0.003213 -0.005344       5        1  6.868457
6      0.011430  0.008703       6        1  6.208673
14    -0.011136 -0.008228       7        1  5.643107
12     0.057764 -0.002308       8        1  5.395153
18    -0.031762  0.026389       9        1  5.028282
5     -0.011653 -0.004394      10        1  4.575684
16     0.075374 -0.017592      11        1  4.490203
19    -0.000986 -0.028363      12        1  4.327262
10     0.008296  0.034705      13        1  3.980828
17    -0.032034 -0.029161      14        1  3.638345
11    -0.030488 -0.001645      15        1  3.561175
7     -0.002229 -0.012525      16        1  3.530705
4     -0.015872  0.021217      17        1  3.010607
13    -0.066755 -0.022180      18        1  2.850693
3      0.010020 -0.074394      19        1  2.674890
0     -0.014485  0.069248      20        1  2.383558, topic_info=    Category         Freq       Term        Total  loglift  logprob
122  Default  1291.000000     health  1291.000000  30.0000  30.0000
637  Default   217.000000     condom   217.000000  29.0000  29.0000
22   Default   417.000000       risk   417.000000  28.0000  28.0000
191  Default   534.000000    patient   534.000000  27.0000  27.0000
26   Default   913.000000      study   913.000000  26.0000  26.0000
..       ...          ...        ...          ...      ...      ...
171  Topic20     8.070173       test   177.188004   0.6475  -5.2077
311  Topic20     8.715142   behavior   322.899292   0.1243  -5.1308
122  Topic20     8.961480     health  1291.506470  -1.2341  -5.1030
213  Topic20     7.858597  predictor   185.619003   0.5745  -5.2343
248  Topic20     7.970245      child   541.455933  -0.4820  -5.2202

[1248 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
706      10  0.477550      1990
546      10  0.471157        90
495       1  0.058682  abortion
495       2  0.058682  abortion
495       3  0.146706  abortion
...     ...       ...       ...
465      15  0.020352     youth
465      16  0.061056     youth
465      17  0.081409     youth
465      19  0.020352     youth
465      20  0.040704     youth

[7749 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 9, 3, 10, 2, 7, 15, 13, 19, 6, 17, 20, 11, 18, 12, 8, 5, 14, 4, 1])